In [26]:
# imports
import models
import utils

import pandas as pd

import numpy as np

from sklearn.preprocessing import MinMaxScaler, Imputer

In [28]:
train_data = utils.load_pickle("./final_features_train2.pkl")
test_data = utils.load_pickle("./final_features_test2.pkl")
# convert from dataframes to arrays
# get train labels and test ids
train_Y = utils.get_train_labels(train_data)
test_ids = test_data[['SK_ID_CURR']]

# drop SK_ID and TARGET
train_data.drop(columns=['TARGET'], inplace=True)
train_data.drop(columns=['SK_ID_CURR'], inplace=True)
test_data.drop(columns=['SK_ID_CURR'], inplace= True)

print(train_data.shape)
print(test_data.shape)

(307511, 606)
(48744, 606)


In [59]:
train_X = train_data.values
test_X = test_data.values
feature_names = train_data.columns # required to create feature importances

# Cleaning dataframes of overflow values
train_X[np.where(train_X >= np.finfo(np.float64).max)] = np.nan
test_X[np.where(test_X >= np.finfo(np.float64).max)] = np.nan

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

imputer.fit(train_X)

train_X = imputer.transform(train_X)
test_X = imputer.transform(test_X)


# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

scaler.fit(train_X)

train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)

In [62]:
import imp; imp.reload(models); imp.reload(utils)
model, predictions, feature_importances, metrics = models.random_forest(train_X, train_Y, test_X, feature_names, n_estimators=500)
utils.create_and_save_submission(test_ids, predictions, save_path='../test_predictions/random_forest.csv')

Training Data Shape:  (307511, 606)
Testing Data Shape:  (48744, 606)


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   43.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  3.5min
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:  8.1min
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:  9.3min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    2.9s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    3.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    8.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   19.3s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:   22.2s finished
[

Predictions saved to:  ../test_predictions/random_forest.csv


In [63]:
print(metrics)

      fold  train     valid
0        0    1.0  0.737297
1        1    1.0  0.735247
2        2    1.0  0.733776
3        3    1.0  0.737842
4        4    1.0  0.736799
5  overall    1.0  0.736175
